In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Conv3D, Flatten, MaxPool2D, AveragePooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.python.keras import metrics
import matplotlib.pyplot as plt
from utilities.rectified_adam import RAdam
import numpy as np
import sys
from utilities.pythonDB import writeToDB, deleteExistingPrimaryKeyDB
from utilities.resnet import ResNet18, ResNet34
%matplotlib inline

Using TensorFlow backend.


In [2]:
def get_random_data(data1, data2, low, high, max_samples=100):
    _, H1, W1, C1 = data1.shape
    _, N = data2.shape
    suff_data1 = np.zeros((max_samples, H1, W1, C1))
    suff_data2 = np.zeros((max_samples, N))
    shuffles = np.random.randint(low, high+1, max_samples)
    for idx in range(shuffles.shape[0]):
        suff_data1[idx] = data1[idx, :, :, :]
        suff_data2[idx] = data2[idx, :]
    return suff_data1, suff_data2

In [3]:
MAX_TRAINING = 25000
MAX_VALIDATION = 5000
MAX_TESTING = 4000
correct_class = {0 : 'airplane', 1 : 'automobile', 2 : 'bird', 3 : 'cat', 4 : 'deer', \
                 5 : 'dog', 6 : 'frog', 7 : 'horse', 8 : 'ship', 9 : 'truck'}

In [4]:
def fetch_selected_data(x_data, y_data):
    new_x_data, new_y_data = np.zeros_like(x_data), np.zeros_like(y_data)
    keep_list = [13, 58, 81, 89]
    curr_x, curr_y = 0, 0
    for i in range(y_data.shape[0]):
        idx = y_data[i][0]
        if idx not in keep_list:
            continue
        # Automobiles
        if idx in [13, 81, 89]:
            new_x_data[curr_x], new_y_data[curr_y]  = x_data[i], np.array([1])
        # Truck
        else:
            new_x_data[curr_x], new_y_data[curr_y]  = x_data[i], np.array([9])
        curr_x, curr_y = curr_x+1, curr_y+1
    return new_x_data[:curr_x], new_y_data[:curr_y]  

In [5]:
#Loading CIFAR-10 dataset
cifar = tf.keras.datasets.cifar100
(x_train_o, y_train_o), (x_test_o, y_test_o) = cifar.load_data()

x_train1, y_train1 = fetch_selected_data(x_train_o, y_train_o)
x_test1, y_test1 = fetch_selected_data(x_test_o, y_test_o)
x_train, y_train = np.concatenate((x_train1, x_test1), axis=0), np.concatenate((y_train1, y_test1), axis=0)

In [2]:
from utilities.data_preprocessors import flip_vertical_np, flip_horizontal_np, rotate_np, flip_rotate, \
perform_swirl_transformation, perform_random_affine_transform, mixed_transformations, add_gaussian_noise, add_sp_noise, \
add_poisson_noise, add_multiplicative_noise, random_image_eraser, correct_low_visibility, gamma_correction

class TransformDataset(object):
    def scale(self, X, x_min=0, x_max=1):
        nom = (X-X.min(axis=0))*(x_max-x_min)
        denom = X.max(axis=0) - X.min(axis=0)
        denom[denom==0] = 1
        X_new = x_min + nom/denom
        return (X_new*255).astype(np.uint8)
    def return_function(self, name, im):
        return getattr(self, 'if_' + name)(im)
    def if_flip_vertical_np(self, im):
        return self.scale(flip_vertical_np(im))
    def if_flip_horizontal_np(self, im):
        return self.scale(flip_horizontal_np(im))
    def if_rotate_np(self, im):
        return self.scale(rotate_np(im))
    def if_flip_rotate(self, im):
        return self.scale(flip_rotate(im))
    def if_perform_swirl_transformation(self, im):
        return self.scale(perform_swirl_transformation(im))
    def if_perform_random_affine_transform(self, im):
        return self.scale(perform_random_affine_transform(im))
    def if_mixed_transformations(self, im):
        return self.scale(mixed_transformations(im))
    def if_add_gaussian_noise(self, im):
        return self.scale(add_gaussian_noise(im))
    def if_add_sp_noise(self, im):
        return self.scale(add_sp_noise(im))
    def if_add_poisson_noise(self, im):
        return self.scale(add_poisson_noise(im))
    def if_add_multiplicative_noise(self, im):
        return self.scale(add_multiplicative_noise(im))
    def if_random_image_eraser(self, im):
        return self.scale(random_image_eraser(im))
    def if_correct_low_visibility(self, im):
        return self.scale(correct_low_visibility(im))
    def if_gamma_correction(self, im):
        return self.scale(gamma_correction(im))

def get_random_shuffle_set(data1, data2, ratios=None):
    if ratios is None:
        ratios = {"flip_vertical_np":.1, 'flip_horizontal_np':.1, 'rotate_np':.15, 'flip_rotate':.2, \
                  'perform_swirl_transformation':.1, 'perform_random_affine_transform':.15, 'mixed_transformations':.2}
    _, H1, W1, C1 = data1.shape
    N, _ = data2.shape
    shuffles = np.random.randint(0, N, N)
    low = 0
    x_train_ret, y_train_ret, img_idx  = np.zeros((N, H1, W1, C1)), np.zeros((N, 1)), 0

    for key in ratios.keys():
        high = int(ratios.get(key) * N)
        data = shuffles[low:low+high]
        low += high
        for idxs in data:
            x_train_ret[img_idx] = TransformDataset().return_function(key, data1[idxs])
            y_train_ret[img_idx] = data2[idxs]
            img_idx += 1
    return x_train_ret, y_train_ret

In [8]:
ratio1 = {"flip_rotate": 0.20, "rotate_np":0.20, "mixed_transformations":.20, 'perform_random_affine_transform':.4}
ratio2 = {"flip_vertical_np": 0.2, "flip_horizontal_np": 0.2, "rotate_np":.3, 'flip_rotate':.3}
ratio3 = {'perform_random_affine_transform': .4, 'perform_swirl_transformation': .4, 'flip_rotate': .2}

x_train_ret1, y_train_ret1 = get_random_shuffle_set(x_train, y_train, ratio1)
x_train_ret2, y_train_ret2 = get_random_shuffle_set(x_train, y_train, ratio2)
x_train_ret3, y_train_ret3 = get_random_shuffle_set(x_train, y_train, ratio3)


final_x_train = np.concatenate((x_train, x_train_ret1, x_train_ret2, x_train_ret3), axis=0)
final_y_train = np.concatenate((y_train, y_train_ret1, y_train_ret2, y_train_ret3), axis=0)

np.save('../saved_numpy_data/cifar100_augmented_x_train.npy', final_x_train)
np.save('../saved_numpy_data/cifar100_augmented_y_train.npy', final_y_train)